In [24]:
## imports here !!!
import numpy as np
import pandas as pd
import xgboost as xgb


from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

from sklearn.impute import SimpleImputer
import random
random.seed(10)

In [19]:
'''
Importing data
'''
train_data= pd.read_csv('iith_foml_2020_train.csv')
train = pd.DataFrame(train_data)
test_data= pd.read_csv('iith_foml_2020_test.csv')
test = pd.DataFrame(test_data)
test.head()


,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,...,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete),Feature 24
0,146,12,42,14,7,1,1,1,118.004,35693.5,...,4200.3,1900.1,NaN,NaN,44,12,2,42,146,37384.5
1,35,0,12,5,0,0,1,0,0.001,471.5,...,531.4,NaN,NaN,NaN,17,6,8,0,35,41465.1
2,1018,8,259,2,1,1,1,1,NaN,35774.5,...,5514.2,NaN,NaN,15.04,1,1,3,20,1018,37826.2
3,383,7,117,5,1,1,1,1,53.002,34094.1,...,3358.4,1100.2,14000.001,15.04,101,6,3,20,383,40277.3
4,1216,7,40,5,2,0,1,4,0.005,1471.3,...,225.1,NaN,NaN,3.02,276,6,7,43,1221,28419.5


### Analysis of missing values...


In [20]:

def Summarise_missing_values(new_data):
    cols_with_missing = (col for col in new_data.columns if new_data[col].isnull().any())
    num_missing = (new_data[cols_with_missing].isnull()).sum()
    print("Missing values in percent")
    # report the results
    print(round((num_missing/new_data.shape[0]) * 100, 2))
    print("######################################################################################") 

print("****************************For TRAIN data")
Summarise_missing_values(train.copy())
print("****************************For Test data")
Summarise_missing_values(test.copy())


****************************For TRAIN data
Missing values in percent
Feature 9      1.41
Feature 10     0.10
Feature 11     0.10
Feature 12     0.10
Feature 13     0.10
Feature 14     0.10
Feature 15     7.24
Feature 16    67.30
Feature 17    54.93
Feature 18    33.20
Feature 24     0.10
dtype: float64
######################################################################################
****************************For Test data
Missing values in percent
Feature 9      0.94
Feature 14     0.47
Feature 15     7.28
Feature 16    65.49
Feature 17    52.82
Feature 18    26.76
dtype: float64
######################################################################################


###  Filling missing values by imputation through mean...

In [21]:

def imputing_missing_values(df):
    data = df.to_numpy()
    # define the imputer
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    imputer  = imputer.fit(data)
    # transform the dataset
    transformed_values = imputer.transform(data)
    imputed_data = pd.DataFrame(data = transformed_values,columns = df.columns) 
    return imputed_data


imputed_train = imputing_missing_values(train)
imputed_test = imputing_missing_values(test)


imputed_test.head()

,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,...,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete),Feature 24
0,146.0,12.0,42.0,14.0,7.0,1.0,1.0,1.0,118.004000,35693.5,...,4200.3,1900.100000,4046.866647,1560.762917,44.0,12.0,2.0,42.0,146.0,37384.5
1,35.0,0.0,12.0,5.0,0.0,0.0,1.0,0.0,0.001000,471.5,...,531.4,1229.219048,4046.866647,1560.762917,17.0,6.0,8.0,0.0,35.0,41465.1
2,1018.0,8.0,259.0,2.0,1.0,1.0,1.0,1.0,5.037701,35774.5,...,5514.2,1229.219048,4046.866647,15.040000,1.0,1.0,3.0,20.0,1018.0,37826.2
3,383.0,7.0,117.0,5.0,1.0,1.0,1.0,1.0,53.002000,34094.1,...,3358.4,1100.200000,14000.001000,15.040000,101.0,6.0,3.0,20.0,383.0,40277.3
4,1216.0,7.0,40.0,5.0,2.0,0.0,1.0,4.0,0.005000,1471.3,...,225.1,1229.219048,4046.866647,3.020000,276.0,6.0,7.0,43.0,1221.0,28419.5


In [15]:
'''
Dividing into X, Y
'''
Y = imputed_train['Target Variable (Discrete)']
X = imputed_train.drop(columns=['Feature 2 (Discrete)','Feature 4 (Discrete)','Feature 9','Feature 20 (Discrete)', 'Feature 21 (Discrete)','Feature 22 (Discrete)','Feature 16', 'Feature 17','Target Variable (Discrete)'],axis = 1)
X_test = imputed_test.drop(columns=['Feature 2 (Discrete)','Feature 4 (Discrete)','Feature 9','Feature 20 (Discrete)', 'Feature 21 (Discrete)','Feature 22 (Discrete)','Feature 16', 'Feature 17'],axis = 1)
#################################################################################

### For 109...

In [25]:
'''
Dividing into X, Y
'''
Y = imputed_train['Target Variable (Discrete)']
X = imputed_train.drop(columns=['Feature 16', 'Feature 17','Target Variable (Discrete)'],axis = 1)
X_test = imputed_test.drop(columns=['Feature 16', 'Feature 17'],axis = 1)
#################################################################################

In [26]:

'''
Training with stratified K-fold
'''

i=1
kf = StratifiedKFold(n_splits=10,random_state=1,shuffle=True)

param_grid = [{'min_child_weight': [2.0]}] #set of trial values for min_child_weight


for train_index,test_index in kf.split(X,Y):
    print('\n{} of kfold {}'.format(i,kf.n_splits))
    xtr,xvl = X.loc[train_index],X.loc[test_index]
    ytr,yvl = Y[train_index],Y[test_index]

    model =GridSearchCV(XGBClassifier(), param_grid, cv=10, scoring= 'f1_macro',iid=True)
    model.fit(xtr, ytr)

    print (model.best_params_)
    y_pred=model.predict(xvl)
    y_pred = [round(value) for value in y_pred]
    
    
    print('accuracy_score',accuracy_score(yvl,y_pred) * 100)
    i+=1


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)



1 of kfold 10


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'min_child_weight': 2.0}
accuracy_score 90.0

2 of kfold 10


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


...

{'min_child_weight': 2.0}
accuracy_score 89.8989898989899

9 of kfold 10


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'min_child_weight': 2.0}
accuracy_score 91.91919191919192

10 of kfold 10


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'min_child_weight': 2.0}
accuracy_score 89.8989898989899


In [27]:
def predict_and_submission(test_data,model,title):### test_data in pd format
    test = pd.DataFrame(test_data)
    predictions = model.predict(test)

    output_test_data = pd.DataFrame() 
    output_test_data['Category'] = predictions.astype(int)

    output_test_data['Id'] = list(np.arange(1,predictions.size+1))

    submission = output_test_data[['Id','Category']]
    submission.to_csv(title, index=False)
    submission.head()
    return submission

##### Plsss change the title........
output = predict_and_submission(X_test,model,title = "S_109.csv")
output.head()

,Id,Category
0,1,6
1,2,0
2,3,1
3,4,1
4,5,1


# Results...

## This model outputs a CSV file titled S003_final.csv